# [ГОСТ34.13-2018](https://files.stroyinf.ru/Data/705/70511.pdf) Режимы работы блочных шифров

## Вспомогательные опреации

*Дополнение сообщения:*
- **Процедура 1**: Пусть P - исходное сообщение, r = |P|mod(l)
- **Процедура 2**: $P^{*}=P||1||0^{l-r-1}$
- **Процедура 3**: Если r = l, то последний блок не изменяется; если r < l, то применяется процедура 2.

*Выработка начального значения*  

В некоторых режимах работы используются величины, начальное значение которых вычисляется на основании IV (синхропосылки). Пусть m – суммарная длина указанных величин. Процедурой инициализации называется процедура выработки начального значения: $I_{m}:V_{|IV|}→V_{m}$. Процедура инициализации считается тривиальной, если $I_{|IV|}=IV$. Если не оговорено иное, то считается, что используется тривиальная процедура инициализации на основе IV необходимой длины.  

Процедура выработки синхропосылки должна удовлетворять одному из следующих требований:
1) значения синхропосылки для режимов простой замены с зацеплением и гаммирования с обратной связью по шифртексту необходимо выбирать случайно, равновероятно и независимо друг от друга из множества всех допустимых значений. В этом случае значение каждой используемой синхропосылки  IV должно быть непредсказуемым (случайным или псевдослучайным): зная значения всех других используемых синхропосылок, значение IV нельзя определить с вероятностью большей, чем $2^{-|IV|}$.  
2) все значения синхропосылок, выработанных для зашифрования на одном и том же ключе в режиме гаммирования, должны быть уникальными, т. е. попарно различными. Для выработки значений синхропосылок может быть использован детерминированный счетчик.  
3) значение синхропосылки для режима гаммирования с обратной связью по выходу должно быть либо непредсказуемым (случайным или псевдослучайным), либо уникальным

*Процедура усечения*  

Применяется для усечения строк длины n до строк длины $s\leq n$, с использованием функции $I_{S}=MSB_{S}$, где $MSB_{S}$ - отображение, ставящее в соответствие строке $z_{m-1}||...||z_{1}||z_{0}, m\geq s$, строку $z_{m-1}||...||z_{m-s+1}||z_{m-s}$

## Режим простой замены

Длина сообщений должна быть кратна длине базового блока (в случае алгоритма Кузнечик – 128бит, Магма – 64бит). Иначе к исходному сообщению применяется одна из процедур дополнения. Зашифрование (расшифрование) заключается в зашифровании (расшифровании) каждого блока текста с помощью базового алгоритма шифрования.

Продемонстрируем соответствие работы алгоритма в режиме простой замены контрольным примерам положения ГОСТ34.13-2018:  
![jupiter](./examples.png)

In [1]:
from gost34.grasshopper import grasshopper
import binascii

key = binascii.unhexlify('8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef') #мастер-ключ (в hex формате)
gost3412 = grasshopper(key) #создаем экземпляр класса на мастер-ключе

In [2]:
message_1 = '1122334455667700ffeeddccbbaa9988' #открытый текст P1

message_encrypt_1 = gost3412.encryption(binascii.unhexlify(message_1)) #шифруем сообщение

binascii.hexlify(bytearray(message_encrypt_1)).decode('CP866') #шифртекст в hex формате

'7f679d90bebc24305a468d42b9d4edcd'

In [3]:
message_2 = '00112233445566778899aabbcceeff0a' #открытый текст P2

message_encrypt_2 = gost3412.encryption(binascii.unhexlify(message_2)) #шифруем сообщение

binascii.hexlify(bytearray(message_encrypt_2)).decode('CP866') #шифртекст в hex формате

'b429912c6e0032f9285452d76718d08b'

In [4]:
message_3 = '112233445566778899aabbcceeff0a00' #открытый текст P3

message_encrypt_3 = gost3412.encryption(binascii.unhexlify(message_3)) #шифруем сообщение

binascii.hexlify(bytearray(message_encrypt_3)).decode('CP866') #шифртекст в hex формате

'f0ca33549d247ceef3f5a5313bd4b157'

In [5]:
message_4 = '2233445566778899aabbcceeff0a0011' #открытый текст P4

message_encrypt_4 = gost3412.encryption(binascii.unhexlify(message_4)) #шифруем сообщение

binascii.hexlify(bytearray(message_encrypt_4)).decode('CP866') #шифртекст в hex формате

'd0b09ccde830b9eb3a02c4c5aa8ada98'

## Режим гаммирования (CTR)

**Гаммирование** — это наложение (или снятие при расшифровке сообщений) на открытое (или зашифрованное) сообщение так называемой криптографической гаммы. 
Для зашифрования (расшифрования) каждого отдельного открытого текста на одном ключе используется значение уникальной синхропосылки $IV\in V_{n/2}$  

**Криптографическая гамма** — это последовательность элементов данных, которая вырабатывается блоками длины $0<s\leq n$ путем зашифрования последовательности значений счетчика $CTR_{i}\in V_{n}$ базовым алгоритмом блочного шифрования с последующим усечением.  

**Начальное значение счетчика**: $CTR_{1}=I_{n}(IV)=IV||0^{n/2}$, его размер равен размеру одного блока используемого алгоритма блочного шифрования (Кузнечик/Магма)
**Последующие значения счетчика вырабатываются следующим образом**:
\begin{equation}CTR_{i+1}=Vec_{n}(Int_{n}(CTR_{i})⊞_{n}1)\end{equation} Т.е. двоичное представление вектора $CTR_{i}$ отображается в элемент кольца $Z_{2^{n}}$, складывается в этом кольце с 1 и обратно отображается в двоичный вектор.

### Зашифрование

Открытый текст P представляется в виде $P=P_{1}||P_{2}||...P_{q}$
Блоки шифртекста вычисляются по следующему правилу:
\begin{equation*}\begin{cases}C_{i}=P_{i}⨁T_{s}(e_{K}(CTR_{i})), i=1,2,...,q-1\\
C_{q}=P_{q}⨁T_{r}(e_{k}(CTR_{q}))\end{cases}\end{equation*}  
Где $e_{K}$ – базовый алгоритм шифрования на ключе K, $T_{i}$ – операция усечения блоков гаммы шифра. В большинстве случаев размер блока исходного текста принимается равным размеру блока используемого алгоритма блочного шифрования поэтому процедура усечения блока гаммы может понадобиться только для последнего блока исходного текста $T_{r}$, в случае, когда общая длина сообщения не кратна размеру одного блока и последний блок получается неполный.  
![jupiter](./gamma.png)  
Результирующий шифртекст имеет вид: $C=C_{1}||C_{2}||...||C_{q}$  
![jupiter](./gamma_encrypt.png)

### Дешифрование

Дешифровка полностью повторяет процесс шифрования за исключением
того, что на вход подают зашифрованный текст, а на выходе получают
расшифрованный, который совпадает с исходным.

**Задание**: реализовать работу блочного шифра в режиме гаммирования.  